## Calculate Days of Sale at Plant Material Combination.

Author: Syamanthaka B

Created: May 2019

----------------------------------------------------------------------------------------------------------

* Input 1 is inventory master data with plant, material and closing stock.
* Input 2 is price per unit for a plant material combination

The code below wrangles these data sets to get a 3 month forward looking Days of Sale or DOS.
DOS is calculated as total cost of inventory / number of days. 

Total cost of inventory is taken over forward looking 3 months, eg. for the month of Jan, forward looking 3 months would include Feb, Mar and Apr. Similar approach is taken for number of days. 

Note: Currently the calculating the total cost over upcoming 3 months takes a bit of computing time. WIP on time optimizations

#### Necessary imports

In [1]:
import pandas as pd
from datetime import datetime as dt
import time
import numpy as np

#### Open input 1 and clean up

In [2]:
inventory_raw = pd.read_excel("INV_BEST - Backup.xlsx", sheet_name="Sheet1")
inventory_raw.rename(columns=lambda x: x.strip(), inplace=True)
inventory_raw = inventory_raw[['Plnt', 'Material', 'From Date', 'Opening Stock', 'Total Receipt Qties', 
                               'Total Issue Quantities', 'Closing Stock']]
inventory_raw = inventory_raw[~inventory_raw['Plnt'].isnull()] ## Some extra cleaning found coincidentally
inventory_raw.dropna()
inventory_raw['Material'] = inventory_raw['Material'].str.strip()
inventory_raw.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock
0,NL92,032210700233,2018-08-01,27.000,8.0,-5.00,30.000
1,NL92,062236202002,2018-08-01,91.000,17.0,-12.00,96.000
2,NL92,0722237002,2018-08-01,0.000,2.0,-2.00,0.000
3,NL92,082201115003,2018-08-01,127.440,0.0,-0.10,127.340
4,NL92,082201165013,2018-08-01,148.675,0.0,-0.08,148.595


#### Add new column for month

In [3]:
inventory_raw['Month'] = pd.DatetimeIndex(inventory_raw['From Date']).month
inventory_raw.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month
0,NL92,032210700233,2018-08-01,27.000,8.0,-5.00,30.000,8
1,NL92,062236202002,2018-08-01,91.000,17.0,-12.00,96.000,8
2,NL92,0722237002,2018-08-01,0.000,2.0,-2.00,0.000,8
3,NL92,082201115003,2018-08-01,127.440,0.0,-0.10,127.340,8
4,NL92,082201165013,2018-08-01,148.675,0.0,-0.08,148.595,8


In [4]:
## Some cosmetic edits
for i in range(3,7):
    inventory_raw.iloc[:,i] = inventory_raw.iloc[:,i].apply(lambda x: round(x))
inventory_raw.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month
0,NL92,032210700233,2018-08-01,27,8,-5,30,8
1,NL92,062236202002,2018-08-01,91,17,-12,96,8
2,NL92,0722237002,2018-08-01,0,2,-2,0,8
3,NL92,082201115003,2018-08-01,127,0,0,127,8
4,NL92,082201165013,2018-08-01,149,0,0,149,8


#### Open input 2 and process

In [5]:
std_price_raw = pd.read_excel("STD_PRICE.xlsx", sheet_name="Sheet1")
std_price_raw.rename(columns=lambda x: x.strip(), inplace=True)
std_price_raw.head()

,Plnt,Material,Material Description,KEY,Mat_Type,MRPpr,ProcType,STD_PRICE
0,NL92,0322-107-00233,WIRE BRAID 14AWG CU/SN,0322-107-00233:NL92,HAWA,BHWA,F,6.24
1,NL92,0622-362-02002,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,0622-362-02002:NL92,FERT,BHWA,F,75.32
2,NL92,0722-237-002,2LG B-CAB. FD12 MID,0722-237-002:NL92,ZMAT,BZMT,E,999999.00
3,NL92,0822-011-15003,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",0822-011-15003:NL92,HAWA,BHWA,F,4.21
4,NL92,0822-011-65013,SH-SLEEVE RNF-3000-12/4-X-SP,0822-011-65013:NL92,HAWA,PMC1,F,3.92


In [6]:
## Clean up
std_price_raw['Material'] = std_price_raw['Material'].str.replace('-', '')
std_price_raw.head()

,Plnt,Material,Material Description,KEY,Mat_Type,MRPpr,ProcType,STD_PRICE
0,NL92,032210700233,WIRE BRAID 14AWG CU/SN,0322-107-00233:NL92,HAWA,BHWA,F,6.24
1,NL92,062236202002,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,0622-362-02002:NL92,FERT,BHWA,F,75.32
2,NL92,0722237002,2LG B-CAB. FD12 MID,0722-237-002:NL92,ZMAT,BZMT,E,999999.00
3,NL92,082201115003,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",0822-011-15003:NL92,HAWA,BHWA,F,4.21
4,NL92,082201165013,SH-SLEEVE RNF-3000-12/4-X-SP,0822-011-65013:NL92,HAWA,PMC1,F,3.92


In [7]:
## Selecting required columns
std_price_raw = std_price_raw.drop_duplicates()
std_price_tmp = std_price_raw[['Plnt', 'Material', 'STD_PRICE', 'Material Description', 'Mat_Type']]
std_price_tmp.head()

,Plnt,Material,STD_PRICE,Material Description,Mat_Type
0,NL92,032210700233,6.24,WIRE BRAID 14AWG CU/SN,HAWA
1,NL92,062236202002,75.32,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,FERT
2,NL92,0722237002,999999.00,2LG B-CAB. FD12 MID,ZMAT
3,NL92,082201115003,4.21,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",HAWA
4,NL92,082201165013,3.92,SH-SLEEVE RNF-3000-12/4-X-SP,HAWA


#### Merge the two inputs

In [8]:
inventory_df = pd.merge(inventory_raw, std_price_tmp, how="left", on=['Plnt', 'Material'])
inventory_df.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month,STD_PRICE,Material Description,Mat_Type
0,NL92,032210700233,2018-08-01,27,8,-5,30,8,6.24,WIRE BRAID 14AWG CU/SN,HAWA
1,NL92,062236202002,2018-08-01,91,17,-12,96,8,75.32,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,FERT
2,NL92,0722237002,2018-08-01,0,2,-2,0,8,999999.00,2LG B-CAB. FD12 MID,ZMAT
3,NL92,082201115003,2018-08-01,127,0,0,127,8,4.21,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",HAWA
4,NL92,082201165013,2018-08-01,149,0,0,149,8,3.92,SH-SLEEVE RNF-3000-12/4-X-SP,HAWA


#### Calculate total cost of closing stock

In [9]:
inventory_df['Total_cost'] = inventory_df['Closing Stock'] * inventory_df['STD_PRICE']
inventory_df.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month,STD_PRICE,Material Description,Mat_Type,Total_cost
0,NL92,032210700233,2018-08-01,27,8,-5,30,8,6.24,WIRE BRAID 14AWG CU/SN,HAWA,187.20
1,NL92,062236202002,2018-08-01,91,17,-12,96,8,75.32,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,FERT,7230.72
2,NL92,0722237002,2018-08-01,0,2,-2,0,8,999999.00,2LG B-CAB. FD12 MID,ZMAT,0.00
3,NL92,082201115003,2018-08-01,127,0,0,127,8,4.21,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",HAWA,534.67
4,NL92,082201165013,2018-08-01,149,0,0,149,8,3.92,SH-SLEEVE RNF-3000-12/4-X-SP,HAWA,584.08


#### Function for forward looking cost of consumption. 
Round 1 of performance optimization reduced time by 50%. Has scope for further improvement

In [12]:
##
### The logic:
# Aggregating by Plnt and Material columns, one needs to take forward looking 3 months of cost.
# For eg for month 3, forward looking 3 months is 4,5,6. 
# For this, then need to sum the total cost
def calc_3magg(row):
    month_no = row['Month']
    plant = row['Plnt']
    mat = row['Material']
    
    np3 = [month_no + 1, month_no + 2, month_no + 3]
    temp_df = inventory_df[(inventory_df['Plnt'] == plant) & (inventory_df['Material'] == mat)]
    
    m3sum = temp_df[temp_df['Month'].isin(np3)]['Total_cost'].agg('sum')
    #m3sum = inventory_pm.loc[(inventory_pm['Plnt'] == plant) & (inventory_pm['Material'] == mat) & (inventory_pm['Month'].isin(np3))]['Total_cost'].agg('sum')
   
    return (m3sum)
    
start = time.time()
#temp['Agg_sum'] = temp.apply(calc_3magg, axis=1)
inventory_df['Agg_sum'] = inventory_df.apply(calc_3magg, axis=1)
end = time.time()
print(end - start)
inventory_df.head()
#temp.head()

904.3926692008972


NameError: name 'inventory_pm' is not defined

#### Creating a month to no. of days reference df

In [14]:
month_data = [[1,31], [2,28], [3,31], [4,30], [5,31], [6,30], [7,31], [8,31], [9,30], [10,31], [11,30], [12,31]]
month_day_df = pd.DataFrame(month_data, columns = ['Month', 'NDays'])
month_day_df.head()

,Month,NDays
0,1,31
1,2,28
2,3,31
3,4,30
4,5,31


In [15]:
for idx, row in month_day_df.iterrows():
    np3m = [row.Month + 1, row.Month + 2, row.Month + 3]
    n3days = month_day_df.loc[month_day_df['Month'].isin(np3m)]['NDays'].agg('sum')
    month_day_df.loc[idx,'N3days'] = n3days

lq = [10,11,12]
lastq = month_day_df['Month'].isin(lq)
month_day_df['N3days'][lastq] = 0
month_day_df

C:\Users\320054667\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Month,NDays,N3days
0,1,31,89.0
1,2,28,92.0
2,3,31,91.0
3,4,30,92.0
4,5,31,92.0
5,6,30,92.0
6,7,31,92.0
7,8,31,91.0
8,9,30,92.0
9,10,31,0.0


#### Adding the month and no. of days forward looking to the inventory df

In [16]:
inventory_df = pd.merge(inventory_df, month_day_df, how="left", on=['Month'])
inventory_df.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month,STD_PRICE,Material Description,Mat_Type,Total_cost,Agg_sum,NDays,N3days
0,NL92,032210700233,2018-08-01,27,8,-5,30,8,6.24,WIRE BRAID 14AWG CU/SN,HAWA,187.20,648.96,31,91.0
1,NL92,062236202002,2018-08-01,91,17,-12,96,8,75.32,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,FERT,7230.72,19281.92,31,91.0
2,NL92,0722237002,2018-08-01,0,2,-2,0,8,999999.00,2LG B-CAB. FD12 MID,ZMAT,0.00,0.00,31,91.0
3,NL92,082201115003,2018-08-01,127,0,0,127,8,4.21,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",HAWA,534.67,1604.01,31,91.0
4,NL92,082201165013,2018-08-01,149,0,0,149,8,3.92,SH-SLEEVE RNF-3000-12/4-X-SP,HAWA,584.08,0.00,31,91.0


#### Calculating ADC - Average Daily consumption as ratio of 3 month forward looking cost / 3 month forward looking no. of days

In [17]:
inventory_df['ADC'] = inventory_df['Agg_sum']/inventory_df['N3days']
inventory_df.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month,STD_PRICE,Material Description,Mat_Type,Total_cost,Agg_sum,NDays,N3days,ADC
0,NL92,032210700233,2018-08-01,27,8,-5,30,8,6.24,WIRE BRAID 14AWG CU/SN,HAWA,187.20,648.96,31,91.0,7.131429
1,NL92,062236202002,2018-08-01,91,17,-12,96,8,75.32,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,FERT,7230.72,19281.92,31,91.0,211.889231
2,NL92,0722237002,2018-08-01,0,2,-2,0,8,999999.00,2LG B-CAB. FD12 MID,ZMAT,0.00,0.00,31,91.0,0.000000
3,NL92,082201115003,2018-08-01,127,0,0,127,8,4.21,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",HAWA,534.67,1604.01,31,91.0,17.626484
4,NL92,082201165013,2018-08-01,149,0,0,149,8,3.92,SH-SLEEVE RNF-3000-12/4-X-SP,HAWA,584.08,0.00,31,91.0,0.000000


#### Calculate DOS or Days of Sale as Closing Stock / ADC

In [18]:
inventory_df['DOS'] = inventory_df['Closing Stock'] / inventory_df['ADC']
inventory_df.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month,STD_PRICE,Material Description,Mat_Type,Total_cost,Agg_sum,NDays,N3days,ADC,DOS
0,NL92,032210700233,2018-08-01,27,8,-5,30,8,6.24,WIRE BRAID 14AWG CU/SN,HAWA,187.20,648.96,31,91.0,7.131429,4.206731
1,NL92,062236202002,2018-08-01,91,17,-12,96,8,75.32,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,FERT,7230.72,19281.92,31,91.0,211.889231,0.453067
2,NL92,0722237002,2018-08-01,0,2,-2,0,8,999999.00,2LG B-CAB. FD12 MID,ZMAT,0.00,0.00,31,91.0,0.000000,NaN
3,NL92,082201115003,2018-08-01,127,0,0,127,8,4.21,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",HAWA,534.67,1604.01,31,91.0,17.626484,7.205067
4,NL92,082201165013,2018-08-01,149,0,0,149,8,3.92,SH-SLEEVE RNF-3000-12/4-X-SP,HAWA,584.08,0.00,31,91.0,0.000000,inf


#### Final cleaning

In [29]:
inventory_df['DOS'].replace([np.inf], 100000000, inplace=True)
inventory_df['DOS'].replace([np.nan], 0, inplace=True)
#inventory_pm[inventory_pm['N3days'] == 0]

In [22]:
inventory_df['DOS'] = inventory_df['DOS'].apply(lambda x: round(x))

#### Add a flag to see if there is demand or not

In [23]:
inventory_df['Demand_flag'] = np.where(inventory_df['ADC'] == 0, 'X', '')
inventory_df.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month,STD_PRICE,Material Description,Mat_Type,Total_cost,Agg_sum,NDays,N3days,ADC,DOS,Demand_flag
0,NL92,032210700233,2018-08-01,27,8,-5,30,8,6.24,WIRE BRAID 14AWG CU/SN,HAWA,187.20,648.96,31,91.0,7.131429,4,
1,NL92,062236202002,2018-08-01,91,17,-12,96,8,75.32,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,FERT,7230.72,19281.92,31,91.0,211.889231,0,
2,NL92,0722237002,2018-08-01,0,2,-2,0,8,999999.00,2LG B-CAB. FD12 MID,ZMAT,0.00,0.00,31,91.0,0.000000,0,X
3,NL92,082201115003,2018-08-01,127,0,0,127,8,4.21,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",HAWA,534.67,1604.01,31,91.0,17.626484,7,
4,NL92,082201165013,2018-08-01,149,0,0,149,8,3.92,SH-SLEEVE RNF-3000-12/4-X-SP,HAWA,584.08,0.00,31,91.0,0.000000,100000000,X


In [28]:
## Save file
path = r"C:\Users\--\DOS_calc.xlsx"
writer = pd.ExcelWriter(path, engine='xlsxwriter')
inventory_df.to_excel(writer, sheet_name='Sheet1')
writer.save()
writer.close()